# مشروع استخراج البيانات (Data Mining Project)

## المقدمة
هذا المشروع يهدف إلى تطبيق تقنيات استخراج البيانات على مجموعة بيانات المبيعات. سنقوم بتنفيذ عدة خوارزميات وتقييم أدائها.

In [ ]:
# استيراد المكتبات اللازمة
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.metrics import accuracy_score, classification_report

# تعيين نمط المخططات
plt.style.use('seaborn')
sns.set_palette('husl')

## الخطوة 1: تحميل البيانات

In [ ]:
# تحميل البيانات
df = pd.read_csv('data/train.csv')
print('شكل البيانات:', df.shape)
df.head()

## الخطوة 2: معالجة البيانات

### تنظيف البيانات

In [ ]:
# فحص القيم المفقودة
print('القيم المفقودة في كل عمود:')
print(df.isnull().sum())

# إزالة السجلات المكررة
df = df.drop_duplicates()
print('\nشكل البيانات بعد إزالة التكرار:', df.shape)

# معالجة التواريخ
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# إضافة ميزات جديدة
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['ShippingDays'] = (df['Ship Date'] - df['Order Date']).dt.days

## الخطوة 3: استكشاف البيانات

In [ ]:
# تحليل المبيعات الشهرية
plt.figure(figsize=(12, 6))
monthly_sales = df.groupby(['Year', 'Month'])['Sales'].sum().reset_index()
monthly_sales['Date'] = pd.to_datetime(monthly_sales[['Year', 'Month']].assign(DAY=1))
plt.plot(monthly_sales['Date'], monthly_sales['Sales'])
plt.title('اتجاه المبيعات الشهرية')
plt.xlabel('التاريخ')
plt.ylabel('إجمالي المبيعات')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# توزيع المبيعات حسب الفئة
plt.figure(figsize=(10, 6))
sns.boxplot(x='Category', y='Sales', data=df)
plt.title('توزيع المبيعات حسب الفئة')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## الخطوة 4: اختيار الميزات

In [ ]:
# تحديد الأعمدة المهمة
features = ['Category', 'Sub-Category', 'Sales', 'Quantity', 'Discount', 'Profit',
            'Segment', 'Region', 'Year', 'Month', 'ShippingDays']
df_selected = df[features]

# تحويل المتغيرات الفئوية
categorical_cols = ['Category', 'Sub-Category', 'Segment', 'Region']
df_encoded = pd.get_dummies(df_selected, columns=categorical_cols)

## الخطوة 5 و 6: تنفيذ وتدريب الخوارزميات

### 1. خوارزمية Apriori

In [ ]:
# إعداد البيانات لتحليل سلة المشتريات
basket = pd.crosstab(df['Order ID'], df['Sub-Category'])
basket = (basket > 0).astype(int)

# تطبيق خوارزمية Apriori
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print('\nTop 5 Association Rules:')
print(rules.sort_values('lift', ascending=False).head())

### 2. شجرة القرار

In [ ]:
# إعداد البيانات للتصنيف
# سنقوم بتصنيف المنتجات إلى فئات ربحية (عالية/منخفضة)
df_encoded['Profit_Category'] = (df_encoded['Profit'] > df_encoded['Profit'].median()).astype(int)

X = df_encoded.drop(['Sales', 'Profit', 'Profit_Category'], axis=1)
y = df_encoded['Profit_Category']

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تدريب نموذج شجرة القرار
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# التنبؤ وتقييم النموذج
dt_pred = dt_model.predict(X_test)
print('\nDecision Tree Results:')
print(classification_report(y_test, dt_pred))

### 3. خوارزمية Naive Bayes

In [ ]:
# تدريب نموذج Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# التنبؤ وتقييم النموذج
nb_pred = nb_model.predict(X_test)
print('\nNaive Bayes Results:')
print(classification_report(y_test, nb_pred))

### 4. خوارزمية K-means

In [ ]:
# إعداد البيانات للتجميع
cluster_features = ['Sales', 'Quantity', 'Discount', 'Profit']
X_cluster = df[cluster_features]

# تطبيع البيانات
scaler = StandardScaler()
X_cluster_scaled = scaler.fit_transform(X_cluster)

# تطبيق خوارزمية K-means
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_cluster_scaled)

# تحليل النتائج
df['Cluster'] = clusters
cluster_stats = df.groupby('Cluster')[cluster_features].mean()
print('\nCluster Statistics:')
print(cluster_stats)

## الخطوة 7: تقييم النماذج باستخدام Cross-Validation

In [ ]:
# تقييم نموذج شجرة القرار
dt_scores = cross_val_score(dt_model, X, y, cv=5)
print('Decision Tree CV Scores:', dt_scores)
print('Average CV Score:', dt_scores.mean())

# تقييم نموذج Naive Bayes
nb_scores = cross_val_score(nb_model, X, y, cv=5)
print('\nNaive Bayes CV Scores:', nb_scores)
print('Average CV Score:', nb_scores.mean())

## الخلاصة

في هذا المشروع، قمنا بتنفيذ وتقييم أربع خوارزميات مختلفة:

1. **خوارزمية Apriori**: تم استخدامها لاكتشاف قواعد الارتباط بين المنتجات.
2. **شجرة القرار**: تم استخدامها للتنبؤ بفئات الربحية.
3. **Naive Bayes**: تم استخدامها أيضاً للتصنيف وتحليل الأداء.
4. **K-means**: تم استخدامها لتجميع المنتجات بناءً على خصائصها.

كل خوارزمية قدمت رؤى مختلفة حول البيانات وساعدت في فهم أنماط المبيعات والربحية بشكل أفضل.